In [3]:
#Libraries
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.parser import parse
from functools import reduce
from random import choice
from requests import get
from scipy.stats import iqr
from sklearn import preprocessing
from sqlalchemy import create_engine
from zipfile import ZipFile
import gzip
import itertools
import locale
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import re
import requests
import seaborn as sns
#import wget
sns.set()

C:\Users\ellio\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [396]:
import difflib

#cases=[('https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page=2&SortOrder=', 'https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page='+i+'&SortOrder=')]
cases=[]

url1 = 'https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page=2&SortOrder='
url2 = 'https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page=3&SortOrder='

def compare_urls(url1, url2):
    urls = [(url1,url2)]
    for a,b in urls:     
        print('{} => {}'.format(a,b))  
        for i,s in enumerate(difflib.ndiff(a, b)):
            if s[0]==' ': continue
            elif s[0]=='-':
                print(u'Delete "{}" from position {}'.format(s[-1],i))
            elif s[0]=='+':
                print(u'Add "{}" to position {}'.format(s[-1],i))    
        print() 
        
compare_urls(url1, url2)

https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page=2&SortOrder= => https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page=3&SortOrder=
Delete "2" from position 218
Add "3" to position 219



In [508]:
with open('./FL_zips.html', 'r') as f:
    html_string = f.read()

In [509]:
html_soup = BeautifulSoup(html_string, 'html.parser')
html_soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>ZIP Code List for Florida</title>
<meta content="Find zip code by searching by city, state, or zip code" name="description"/>
<meta content="city zip codes county state us usa zipcodes by state" name="keywords"/>
<style type="text/css">
  		body {
			font-size: 16px;
			font-family:Arial;
    	}
		.small {
			font-size: 12px; /* 75% of the baseline */	
		}
		.large {
			font-size: 20px; /* 125% of the baseline */
		}
		.bold {
			font-weight:bold;	
		}
		.inner_table {
			padding: 1px;
			border-spacing: 0px;
			width: 100%;
		}
		inner_table tr {
			border: 0px;
			padding: 0px;
		}
		.inner_table td {
			padding: 3px;
			border: grey solid 1px;
		}
		
		tr:nth-child(even) {
			background-color: #FFFFCC;
		}
		tr:nth-child(odd) {
			background-color: #FFFFFF;
		}
		#header img {
			display: none;
		}
		
		.navigation {
			margin: 0px;
			paddin

In [511]:
house_containers = html_soup.find_all('table', class_="inner_table")


In [521]:
house_containers[0]

<table class="inner_table">
<tr bgcolor="#FFFFFF">
<td colspan="4">
<div align="center">
<p class="white">Zip Codes for the State of<br/>
<strong>
<a href="//www.zipcodestogo.com/Florida/">Florida</a>
</strong>
</p>
</div>
</td>
</tr>
<tr style="background-color: #F5F5F5;">
<td width="70"><div align="center"><strong>Zip Code</strong></div></td>
<td width="128"><strong>City</strong></td>
<td width="109"><strong>County</strong></td>
<td width="107"><div align="center"><strong>Zip Code Map</strong></div></td>
</tr>
<tr>
<td align="center"><a href="//www.zipcodestogo.com/Orange Park/FL/32003/">32003</a></td>
<td>Orange Park</td>
<td><a href="/Clay/FL/">Clay</a></td>
<td><div align="center"><a href="//maps.google.com/maps?oi=map&amp;q=32003">View Map</a></div></td>
</tr>
<tr>
<td align="center"><a href="//www.zipcodestogo.com/Ponte Vedra Beach/FL/32004/">32004</a></td>
<td>Ponte Vedra Beach</td>
<td><a href="/Saint johns/FL/">Saint Johns</a></td>
<td><div align="center"><a href="//maps.goog

In [518]:
len(house_containers)

2

In [519]:
len(house_containers[0])

2949

In [522]:
z = house_containers[0]

In [525]:
z.find_all('a', href="//www.zipcodestogo.com/Bryceville/FL/32009/")

[]

In [537]:
zz = z.find_all('td', align="center")
zz[9]

<td align="center"><a href="//www.zipcodestogo.com/Lake City/FL/32025/">32025</a></td>

In [538]:
len(zz)

1472

In [543]:
zip_list = []
for i in range(len(zz)):
    zip_list.append(str(zz[i]))



In [545]:
df_zip = pd.DataFrame(zip_list)

In [560]:
df_zip.head()

,0,CITY,ZIP
0,"<td align=""center""><a href=""//www.zipcodestogo...",Orange Park,
1,"<td align=""center""><a href=""//www.zipcodestogo...",Ponte Vedra Beach,
2,"<td align=""center""><a href=""//www.zipcodestogo...",Fleming Island,
3,"<td align=""center""><a href=""//www.zipcodestogo...",Bostwick,
4,"<td align=""center""><a href=""//www.zipcodestogo...",Branford,


In [556]:
def clean_zip(str_zip):
    try:
        front = re.sub('<td align="center"><a href="//www.zipcodestogo.com/', '', str_zip)

        return re.sub(r'\WFL\W\d{5}\W\W\W\d{5}\W\Wa\W\W\Wtd\W$', '', front)
        
    except:
        return ""

In [557]:
df_zip['CITY'] = df_zip[0].apply(clean_zip)

In [561]:
#22
def clean_zip2(str_zip):
    try:
        #front = re.sub('<td align="center"><a href="//www.zipcodestogo.com/', '', str_zip)
        front = str_zip[-22:]
        return re.sub(r'\W\W\W\d{5}\W\Wa\W\W\Wtd\W$', '', front)
        
    except:
        return ""

In [562]:
df_zip['ZIP'] = df_zip[0].apply(clean_zip2)

In [563]:
df_zip.head()

,0,CITY,ZIP
0,"<td align=""center""><a href=""//www.zipcodestogo...",Orange Park,32003
1,"<td align=""center""><a href=""//www.zipcodestogo...",Ponte Vedra Beach,32004
2,"<td align=""center""><a href=""//www.zipcodestogo...",Fleming Island,32006
3,"<td align=""center""><a href=""//www.zipcodestogo...",Bostwick,32007
4,"<td align=""center""><a href=""//www.zipcodestogo...",Branford,32008


In [ ]:
for elem in flat_ac:
    #emails.append(re.search(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', str(elem.find_all('div'))))
    name = elem.find_all('a', class_="ao_results_icon_text A detail-page")
    email = re.search(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', str(elem.find_all('div')))
    phones = elem.find_all('div', class_="ao-phone")
    address = elem.find_all('div', class_="ao-address")
    details.append([name, email, phones, address])

In [390]:

import math

cases = math.ceil(1605/50)

cases

33

In [16]:
import time

In [32]:
#keep

urlz=[]
results_dict = {}

for i in range(1,cases+1):
    url = 'https://www.bhhsfloridarealty.com/AgentSearch/Results.aspx?SearchType=agent&FirstName=&LastName=&OfficeName=&Address=&City=&State=&Country=-32768&Zip=&Languages=&Titles=&Specialties=&Accreditations=&Areas=&rpp=50&page='+str(i)+'&SortOrder='
    urlz.append(url)

In [34]:
import math

cases = math.ceil(1605/50)

results_dict = {}
page = 1

for url in urlz:
    print("Round "+str(page))
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    results_dict[page] = html_soup
    page += 1
    print('waiting...')
    time.sleep(60)
    print('next!!')
    

Round 1
waiting...
next!!
Round 2
waiting...
next!!
Round 3
waiting...
next!!
Round 4
waiting...
next!!
Round 5
waiting...
next!!
Round 6
waiting...
next!!
Round 7
waiting...
next!!
Round 8
waiting...
next!!
Round 9
waiting...
next!!
Round 10
waiting...
next!!
Round 11
waiting...
next!!
Round 12
waiting...
next!!
Round 13
waiting...
next!!
Round 14
waiting...
next!!
Round 15
waiting...
next!!
Round 16
waiting...
next!!
Round 17
waiting...
next!!
Round 18
waiting...
next!!
Round 19
waiting...
next!!
Round 20
waiting...
next!!
Round 21
waiting...
next!!
Round 22
waiting...
next!!
Round 23
waiting...
next!!
Round 24
waiting...
next!!
Round 25
waiting...
next!!
Round 26
waiting...
next!!
Round 27
waiting...
next!!
Round 28
waiting...
next!!
Round 29
waiting...
next!!
Round 30
waiting...
next!!
Round 31
waiting...
next!!
Round 32
waiting...
next!!


In [397]:
lst = [] 

for value in results_dict.values():
    lst.append(value)

In [60]:
## THIS WORKED

import csv

with open('test.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(lst)

In [400]:
all_agent_containers = []

for value in results_dict.values():
    all_agent_containers.append(value.find_all('div', class_="ao-info-container"))

In [401]:
flat_ac = []

for grp in range(len(all_agent_containers)):
    for sub_grp in range(len(all_agent_containers[grp]) ):
        flat_ac.append(all_agent_containers[grp][sub_grp])

In [402]:
len(flat_ac)

1605

In [405]:

emails = []
details = []

In [406]:
for elem in flat_ac:
    name = elem.find_all('a', class_="ao_results_icon_text A detail-page")
    email = re.search(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', str(elem.find_all('div')))
    phones = elem.find_all('div', class_="ao-phone")
    address = elem.find_all('div', class_="ao-address")
    details.append([name, email, phones, address])

In [407]:
df = pd.DataFrame(details)

In [408]:
for col in df.columns:
    df[col+10] = df[col].astype(str)

In [409]:
df

,0,1,2,3,10,11,12,13
0,[[Jay Aamodt]],"<re.Match object; span=(2412, 2429), match='ja...","[[(954) 465-5332], [(954) 465-5332]]","[[\n, [1306 S.E. 17th St.], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2412, 2429), match='ja...","[<div class=""ao-phone"" id=""ao-phone"">(954) 465...","[<div class=""ao-address"">\n<span class=""ao-add..."
1,[[Shari Abbott]],"<re.Match object; span=(2339, 2367), match='sh...","[[(321) 427-3658], [(321) 427-3658]]","[[\n, [2000 Highway A1A], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2339, 2367), match='sh...","[<div class=""ao-phone"" id=""ao-phone"">(321) 427...","[<div class=""ao-address"">\n<span class=""ao-add..."
2,[[Patti Abrams]],"<re.Match object; span=(2330, 2363), match='pa...","[[], []]","[[\n, [6175 Bayview Dr.], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2330, 2363), match='pa...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add..."
3,[[Joe Abreu]],"<re.Match object; span=(2423, 2437), match='ab...","[[(954) 755-1879], [(954) 755-1879]]","[[\n, [2780 University Dr. 3rd Floor], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2423, 2437), match='ab...","[<div class=""ao-phone"" id=""ao-phone"">(954) 755...","[<div class=""ao-address"">\n<span class=""ao-add..."
4,[[Karen Adamopoulos]],"<re.Match object; span=(2808, 2836), match='ka...","[[407-353-9970], [407-353-9970]]","[[\n, [515 W. Morse Boulevard], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2808, 2836), match='ka...","[<div class=""ao-phone"" id=""ao-phone"">407-353-9...","[<div class=""ao-address"">\n<span class=""ao-add..."
...,...,...,...,...,...,...,...,...
1600,[[David Zenga]],"<re.Match object; span=(2413, 2438), match='Da...","[[(772) 240-4998], [(772) 240-4998]]","[[\n, [1430 St. Lucie West Blvd. Ste. 102], \r...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2413, 2438), match='Da...","[<div class=""ao-phone"" id=""ao-phone"">(772) 240...","[<div class=""ao-address"">\n<span class=""ao-add..."
1601,[[Cuiping (Ellen) Zhou]],"<re.Match object; span=(2368, 2387), match='el...","[[(954) 775-6060], [(954) 775-6060]]","[[\n, [822 E. Atlantic Ave.], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2368, 2387), match='el...","[<div class=""ao-phone"" id=""ao-phone"">(954) 775...","[<div class=""ao-address"">\n<span class=""ao-add..."
1602,[[Melinda Zisser]],"<re.Match object; span=(2344, 2359), match='mz...","[[], []]","[[\n, [1306 S.E. 17th St.], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2344, 2359), match='mz...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add..."
1603,[[Jay & Marie Ziv]],"<re.Match object; span=(2443, 2462), match='zi...","[[954-854-0408], [954-854-0408]]","[[\n, [2780 University Dr. 3rd Floor], \r\n ...","[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2443, 2462), match='zi...","[<div class=""ao-phone"" id=""ao-phone"">954-854-0...","[<div class=""ao-address"">\n<span class=""ao-add..."


In [410]:
df2 = df[[10,11,12,13]].copy()

In [411]:
df2

,10,11,12,13
0,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2412, 2429), match='ja...","[<div class=""ao-phone"" id=""ao-phone"">(954) 465...","[<div class=""ao-address"">\n<span class=""ao-add..."
1,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2339, 2367), match='sh...","[<div class=""ao-phone"" id=""ao-phone"">(321) 427...","[<div class=""ao-address"">\n<span class=""ao-add..."
2,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2330, 2363), match='pa...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add..."
3,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2423, 2437), match='ab...","[<div class=""ao-phone"" id=""ao-phone"">(954) 755...","[<div class=""ao-address"">\n<span class=""ao-add..."
4,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2808, 2836), match='ka...","[<div class=""ao-phone"" id=""ao-phone"">407-353-9...","[<div class=""ao-address"">\n<span class=""ao-add..."
...,...,...,...,...
1600,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2413, 2438), match='Da...","[<div class=""ao-phone"" id=""ao-phone"">(772) 240...","[<div class=""ao-address"">\n<span class=""ao-add..."
1601,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2368, 2387), match='el...","[<div class=""ao-phone"" id=""ao-phone"">(954) 775...","[<div class=""ao-address"">\n<span class=""ao-add..."
1602,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2344, 2359), match='mz...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add..."
1603,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2443, 2462), match='zi...","[<div class=""ao-phone"" id=""ao-phone"">954-854-0...","[<div class=""ao-address"">\n<span class=""ao-add..."


In [412]:
df2.columns=['name', 'email', 'phones', 'address']

In [178]:
df2.to_csv('needs_cleaning.csv', index=False, encoding='utf-8')

In [413]:
df2

,name,email,phones,address
0,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2412, 2429), match='ja...","[<div class=""ao-phone"" id=""ao-phone"">(954) 465...","[<div class=""ao-address"">\n<span class=""ao-add..."
1,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2339, 2367), match='sh...","[<div class=""ao-phone"" id=""ao-phone"">(321) 427...","[<div class=""ao-address"">\n<span class=""ao-add..."
2,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2330, 2363), match='pa...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add..."
3,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2423, 2437), match='ab...","[<div class=""ao-phone"" id=""ao-phone"">(954) 755...","[<div class=""ao-address"">\n<span class=""ao-add..."
4,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2808, 2836), match='ka...","[<div class=""ao-phone"" id=""ao-phone"">407-353-9...","[<div class=""ao-address"">\n<span class=""ao-add..."
...,...,...,...,...
1600,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2413, 2438), match='Da...","[<div class=""ao-phone"" id=""ao-phone"">(772) 240...","[<div class=""ao-address"">\n<span class=""ao-add..."
1601,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2368, 2387), match='el...","[<div class=""ao-phone"" id=""ao-phone"">(954) 775...","[<div class=""ao-address"">\n<span class=""ao-add..."
1602,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2344, 2359), match='mz...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add..."
1603,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2443, 2462), match='zi...","[<div class=""ao-phone"" id=""ao-phone"">954-854-0...","[<div class=""ao-address"">\n<span class=""ao-add..."


In [414]:
def clean_email(str_email):
    return re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', str_email )[0]

In [415]:
df2['email_cl'] = df2['email'].apply(clean_email)

In [416]:
def clean_phones(str_phones):
    try:
        return  re.findall(r'\W\d{3}\W\s\d{3}-\d{4}', str_phones )
    except:
        return ""

In [425]:
df2['phones_cl'] = df2['phones'].apply(clean_phones)

In [427]:
df2.head()

,name,email,phones,address,email_cl,phones_cl,phoney,phonex,name_cl,address_cl,phone1,phone2
0,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2412, 2429), match='ja...","[<div class=""ao-phone"" id=""ao-phone"">(954) 465...","[<div class=""ao-address"">\n<span class=""ao-add...",jay@jayaamodt.com,"[(954) 465-5332, (954) 465-5332]",(954) 465-5332,(954) 465-5332,Jay Aamodt,"1306 S.E. 17th St. Fort Lauderdale, Florida 33...",(954) 465-5332,(954) 465-5332
1,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2339, 2367), match='sh...","[<div class=""ao-phone"" id=""ao-phone"">(321) 427...","[<div class=""ao-address"">\n<span class=""ao-add...",shari@shariabbottrealtor.com,"[(321) 427-3658, (321) 427-3658]",(321) 427-3658,(321) 427-3658,Shari Abbott,"2000 Highway A1A Indian Harbour Beach, Florida...",(321) 427-3658,(321) 427-3658
2,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2330, 2363), match='pa...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add...",pattiabrams@bhhsfloridarealty.net,[],None,None,Patti Abrams,"6175 Bayview Dr. Fort Lauderdale, Florida 33308",None,None
3,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2423, 2437), match='ab...","[<div class=""ao-phone"" id=""ao-phone"">(954) 755...","[<div class=""ao-address"">\n<span class=""ao-add...",abreu14@cs.com,"[(954) 755-1879, (954) 755-1879]",(954) 755-1879,(954) 755-1879,Joe Abreu,"2780 University Dr. 3rd Floor Coral Springs, F...",(954) 755-1879,(954) 755-1879
4,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2808, 2836), match='ka...","[<div class=""ao-phone"" id=""ao-phone"">407-353-9...","[<div class=""ao-address"">\n<span class=""ao-add...",karenadamopoulos@outlook.com,[],None,None,Karen Adamopoulos,"515 W. Morse Boulevard Winter Park, Florida 32...",None,None


In [433]:
df2[['phone1','phone2']] = pd.DataFrame(df2['phones_cl'].tolist(), index= df2.index)


In [434]:
def clean_name(str_name):
    try:
        front = re.sub(r'.*AgentSearch\W[a-zA-Z0-9._-]+\W{2}', '', str_name)
        return re.sub(r'\W\Wa\W\W', '', front)
        
    except:
        return ""

In [435]:
df2['name_cl'] = df2['name'].apply(clean_name)

In [436]:
def clean_address(str_address):
    try:
        #return str(str_address).replace('\r\n', "")
        step1 =  str(str_address).replace('[<div class="ao-address">\n<span class="ao-address">', "")
        step2 = str(step1).replace('</span>', "")
        step3 = str(step2).replace('\r\n', "")
        step4 = str(step3).replace("</div>]", "")
        step5 = str(step4).replace("   ", "")
        
        return step5
    except:
        return ""

In [437]:
df2['address_cl'] = df2['address'].apply(clean_address)

In [438]:
df2

,name,email,phones,address,email_cl,phones_cl,phoney,phonex,name_cl,address_cl,phone1,phone2
0,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2412, 2429), match='ja...","[<div class=""ao-phone"" id=""ao-phone"">(954) 465...","[<div class=""ao-address"">\n<span class=""ao-add...",jay@jayaamodt.com,"[(954) 465-5332, (954) 465-5332]",(954) 465-5332,(954) 465-5332,Jay Aamodt,"1306 S.E. 17th St. Fort Lauderdale, Florida 33...",(954) 465-5332,(954) 465-5332
1,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2339, 2367), match='sh...","[<div class=""ao-phone"" id=""ao-phone"">(321) 427...","[<div class=""ao-address"">\n<span class=""ao-add...",shari@shariabbottrealtor.com,"[(321) 427-3658, (321) 427-3658]",(321) 427-3658,(321) 427-3658,Shari Abbott,"2000 Highway A1A Indian Harbour Beach, Florida...",(321) 427-3658,(321) 427-3658
2,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2330, 2363), match='pa...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add...",pattiabrams@bhhsfloridarealty.net,[],None,None,Patti Abrams,"6175 Bayview Dr. Fort Lauderdale, Florida 33308",None,None
3,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2423, 2437), match='ab...","[<div class=""ao-phone"" id=""ao-phone"">(954) 755...","[<div class=""ao-address"">\n<span class=""ao-add...",abreu14@cs.com,"[(954) 755-1879, (954) 755-1879]",(954) 755-1879,(954) 755-1879,Joe Abreu,"2780 University Dr. 3rd Floor Coral Springs, F...",(954) 755-1879,(954) 755-1879
4,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2808, 2836), match='ka...","[<div class=""ao-phone"" id=""ao-phone"">407-353-9...","[<div class=""ao-address"">\n<span class=""ao-add...",karenadamopoulos@outlook.com,[],None,None,Karen Adamopoulos,"515 W. Morse Boulevard Winter Park, Florida 32...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1600,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2413, 2438), match='Da...","[<div class=""ao-phone"" id=""ao-phone"">(772) 240...","[<div class=""ao-address"">\n<span class=""ao-add...",David.Zenga1965@gmail.com,"[(772) 240-4998, (772) 240-4998]",(772) 240-4998,(772) 240-4998,David Zenga,1430 St. Lucie West Blvd. Ste. 102 Port Saint ...,(772) 240-4998,(772) 240-4998
1601,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2368, 2387), match='el...","[<div class=""ao-phone"" id=""ao-phone"">(954) 775...","[<div class=""ao-address"">\n<span class=""ao-add...",ellen.zcp@gmail.com,"[(954) 775-6060, (954) 775-6060]",(954) 775-6060,(954) 775-6060,Cuiping (Ellen) Zhou,"822 E. Atlantic Ave. Delray Beach, Florida 33483",(954) 775-6060,(954) 775-6060
1602,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2344, 2359), match='mz...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add...",mzisser@msn.com,[],None,None,Melinda Zisser,"1306 S.E. 17th St. Fort Lauderdale, Florida 33...",None,None
1603,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2443, 2462), match='zi...","[<div class=""ao-phone"" id=""ao-phone"">954-854-0...","[<div class=""ao-address"">\n<span class=""ao-add...",zivteam21@gmail.com,[],None,None,Jay &amp; Marie Ziv,"2780 University Dr. 3rd Floor Coral Springs, F...",None,None


In [443]:
df2['address_cl'] = df2['address_cl'].str.rstrip()

def ex_zip(add):
    try:
        return re.findall(r'[0-9]{4,5}$', add )
    except:
        return ""

df2['zip'] = df2['address_cl'].apply(ex_zip)

df2[['zip']] = pd.DataFrame(df2['zip'].tolist(), index= df2.index)


In [473]:
df2['zip'] = df2['address_cl'].str[-5:]

In [482]:
df2['address_nozip'] = df2['address_cl'].str[:-5].str.rstrip()

In [484]:
pip install zipcodes

Note: you may need to restart the kernel to use updated packages.


In [485]:
from pprint import pprint
import zipcodes

In [492]:
#pprint(zipcodes.matching('33316'))
zipcodes.matching('33316')[0]['acceptable_cities'][0]

'Ft Lauderdale'

In [497]:
def city(zip):
    return zipcodes.matching(zip)[0]['acceptable_cities']#[0]

In [498]:
df2['city'] = df2['zip2'].apply(city)

In [503]:
df2['add_commas'] = df2['address_cl'].str.count(',')

In [504]:
df2['add_commas'].value_counts()

1    1601
2       4
Name: add_commas, dtype: int64

In [505]:
df2.to_csv('final_clean.csv', index=False, encoding='utf-8')

In [564]:
df2.head()

,name,email,phones,address,email_cl,phones_cl,phoney,phonex,name_cl,address_cl,phone1,phone2,zip,zip2,address_nozip,city,add_commas
0,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2412, 2429), match='ja...","[<div class=""ao-phone"" id=""ao-phone"">(954) 465...","[<div class=""ao-address"">\n<span class=""ao-add...",jay@jayaamodt.com,"[(954) 465-5332, (954) 465-5332]",(954) 465-5332,(954) 465-5332,Jay Aamodt,"1306 S.E. 17th St. Fort Lauderdale, Florida 33316",(954) 465-5332,(954) 465-5332,33316,33316,"1306 S.E. 17th St. Fort Lauderdale, Florida",[Ft Lauderdale],1
1,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2339, 2367), match='sh...","[<div class=""ao-phone"" id=""ao-phone"">(321) 427...","[<div class=""ao-address"">\n<span class=""ao-add...",shari@shariabbottrealtor.com,"[(321) 427-3658, (321) 427-3658]",(321) 427-3658,(321) 427-3658,Shari Abbott,"2000 Highway A1A Indian Harbour Beach, Florida...",(321) 427-3658,(321) 427-3658,32937,32937,"2000 Highway A1A Indian Harbour Beach, Florida","[Ind Hbr Bch, Indian Harbour Beach, Indn Hbr B...",1
2,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2330, 2363), match='pa...","[<div class=""ao-phone"" id=""ao-phone""></div>, <...","[<div class=""ao-address"">\n<span class=""ao-add...",pattiabrams@bhhsfloridarealty.net,[],None,None,Patti Abrams,"6175 Bayview Dr. Fort Lauderdale, Florida 33308",None,None,33308,33308,"6175 Bayview Dr. Fort Lauderdale, Florida","[Ft Lauderdale, Laud By Sea, Lauderdale By The...",1
3,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2423, 2437), match='ab...","[<div class=""ao-phone"" id=""ao-phone"">(954) 755...","[<div class=""ao-address"">\n<span class=""ao-add...",abreu14@cs.com,"[(954) 755-1879, (954) 755-1879]",(954) 755-1879,(954) 755-1879,Joe Abreu,"2780 University Dr. 3rd Floor Coral Springs, F...",(954) 755-1879,(954) 755-1879,33065,33065,"2780 University Dr. 3rd Floor Coral Springs, F...","[Coral Springs, Margate]",1
4,"[<a class=""ao_results_icon_text A detail-page""...","<re.Match object; span=(2808, 2836), match='ka...","[<div class=""ao-phone"" id=""ao-phone"">407-353-9...","[<div class=""ao-address"">\n<span class=""ao-add...",karenadamopoulos@outlook.com,[],None,None,Karen Adamopoulos,"515 W. Morse Boulevard Winter Park, Florida 32789",None,None,32789,32789,"515 W. Morse Boulevard Winter Park, Florida",[],1


In [567]:
df3 = df2.merge(df_zip[['ZIP','CITY']], how='left', left_on='zip', right_on='ZIP')

In [568]:
df3.to_csv('final_clean_CITY.csv', index=False, encoding='utf-8')

In [466]:
df2['address_cl'][0]

'1306 S.E. 17th St. Fort Lauderdale, Florida 33316'

In [467]:
df2['address_cl'][1]

'2000 Highway A1A Indian Harbour Beach, Florida 32937'

In [468]:
df2['address_cl'][2]

'6175 Bayview Dr. Fort Lauderdale, Florida 33308'

In [ ]:
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", "Connecticut", "District ", "of Columbia", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [507]:
pd.DataFrame(state_names).to_csv('states.csv', index=False)

In [344]:
colX = []
for i in df2.columns[-4:]:
    colX.append(i)

In [346]:
df3 = df2[colX].copy()